In [1]:
from haystack.preprocessor.utils import convert_files_to_dicts
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.file_converter.txt import TextConverter
from haystack.file_converter.pdf import PDFToTextConverter
from haystack.file_converter.docx import DocxToTextConverter
from haystack.preprocessor.preprocessor import PreProcessor
from haystack import Finder
from haystack.document_store.memory import InMemoryDocumentStore
from haystack.retriever.sparse import TfidfRetriever
from haystack.retriever.dense import DensePassageRetriever, EmbeddingRetriever
from haystack.pipeline import ExtractiveQAPipeline

03/08/2021 03:35:07 - INFO - faiss -   Loading faiss.
03/08/2021 03:35:08 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
all_docs = convert_files_to_dicts(dir_path="../test_disclosure_docs/")

03/08/2021 03:35:09 - WARNING - haystack.preprocessor.utils -   Skipped file ../test_disclosure_docs/.DS_Store as type  is not supported here. See haystack.file_converter for support of more file types
03/08/2021 03:35:09 - INFO - haystack.preprocessor.utils -   Converting ../test_disclosure_docs/ExxonMobil_2017_Sustainability_Report_Highlights.pdf
03/08/2021 03:35:09 - INFO - haystack.preprocessor.utils -   Converting ../test_disclosure_docs/AT&T_Corporate_Responsibility_Summary_2019-2020.pdf
03/08/2021 03:35:09 - INFO - haystack.preprocessor.utils -   Converting ../test_disclosure_docs/Google_Environmental_Report_2019.pdf
03/08/2021 03:35:10 - INFO - haystack.preprocessor.utils -   Converting ../test_disclosure_docs/Amazon_Sustainability_2020.pdf
03/08/2021 03:35:11 - INFO - haystack.preprocessor.utils -   Converting ../test_disclosure_docs/UnitedHealth_Group_Helping_Build_Healthier_Communities.pdf
03/08/2021 03:35:11 - INFO - haystack.preprocessor.utils -   Converting ../test_disclo

In [3]:
all_docs

[{'text': '2017 Sustainability Report Highlights\n\x0cTable of contents\n\nChairman\'s letter\n\n3\n\nExxonMobil and sustainability\n\n4\n\nExternal Sustainability Advisory Panel statement\n\n6\n\nAddressing the United Nations Sustainable Development Goals\n\n8\n\nCorporate governance\n\n9\n\nSafety, health and the workplace\n\n12\n\nManaging the risks of climate change\n\n16\n\nEnvironmental performance\n\n20\n\nCommunity engagement and human rights\n\n23\n\nLocal development and supply chain management\n\n26\n\nCase study: ExxonMobil\'s response to Hurricane Harvey\n\n29\n\nAbout the Sustainability Report Highlights\n\n30\n\nPerformance data table\n\n31\n\nNote: Exxon Mobil Corporation has numerous affiliates, with many names that include ExxonMobil, Exxon, Mobil, Esso and XTO Energy. For convenience and simplicity, those terms and terms such as corporation, company, our, we and its are sometimes used as abbreviated references to specific affiliates or affiliate groups. ExxonMobil is

In [4]:
import glob
pdfCounter = len(glob.glob1("../test_disclosure_docs","*.pdf"))
pdfCounter

8

In [5]:
name_list = [all_docs[i]["meta"]["name"] for i in range(0,pdfCounter)]
name_list

['ExxonMobil_2017_Sustainability_Report_Highlights.pdf',
 'AT&T_Corporate_Responsibility_Summary_2019-2020.pdf',
 'Google_Environmental_Report_2019.pdf',
 'Amazon_Sustainability_2020.pdf',
 'UnitedHealth_Group_Helping_Build_Healthier_Communities.pdf',
 'Walmart_2019_ESG_Report.pdf',
 'Apple_Environmental_Responsibility_Report_2019.pdf',
 'McKesson_Corporate_Responsibility_Report_2019.pdf']

In [6]:
sust_disc_texts = [all_docs[i]["text"] for i in range(0,pdfCounter)]

for i in range(0,pdfCounter):
    with open(f"/Users/yannusinovich/Documents/Projects/NLP-Workshop/aisc-nlp-project/test_disclosure_docs/{name_list[i]}.txt", "w") as text_file:
        print(f"{sust_disc_texts[i]}", file=text_file)

In [7]:
document_store = InMemoryDocumentStore()

In [8]:
document_store.write_documents(all_docs)

In [9]:
# pick one of three retrievers

retriever = TfidfRetriever(document_store=document_store)


# retriever = DensePassageRetriever(document_store=document_store,
#                                   query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
#                                   passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
#                                   max_seq_len_query=64,
#                                   max_seq_len_passage=256,
#                                   batch_size=16,
#                                   use_gpu=False,
#                                   embed_title=True,
#                                   use_fast_tokenizers=True)
# # Important: 
# # Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# # previously indexed documents and update their embedding representation. 
# # While this can be a time consuming operation (depending on corpus size), it only needs to be done once. 
# # At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
# document_store.update_embeddings(retriever)


# retriever = EmbeddingRetriever(document_store=document_store,
#                                embedding_model="deepset/sentence_bert")
# document_store.update_embeddings(retriever)

03/08/2021 03:35:13 - INFO - haystack.retriever.sparse -   Found 10868 candidate paragraphs from 16 docs in DB


In [10]:
reader = FARMReader(model_name_or_path="deepset/xlm-roberta-large-squad2")
# source: https://huggingface.co/deepset/xlm-roberta-large-squad2
# other models: https://huggingface.co/models?pipeline_tag=question-answering

03/08/2021 03:35:13 - INFO - farm.utils -   Using device: CPU 
03/08/2021 03:35:13 - INFO - farm.utils -   Number of GPUs: 0
03/08/2021 03:35:13 - INFO - farm.utils -   Distributed Training: False
03/08/2021 03:35:13 - INFO - farm.utils -   Automatic Mixed Precision: None
03/08/2021 03:35:32 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
03/08/2021 03:35:32 - INFO - farm.utils -   Using device: CPU 
03/08/2021 03:35:32 - INFO - farm.utils -   Number of GPUs: 0
03/08/2021 03:35:32 - INFO - farm.utils -   Distributed Training: False
03/08/2021 03:35:32 - INFO - farm.utils -   Automatic Mixed Precision: None
03/08/2021 03:35:32 - INFO - farm.infer -   Got ya 15 parallel workers to do inference ...
03/08/2021 03:35:32 - INFO - farm.infer -    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
03/08/2021 03:35:32 - INFO - farm.infer -   /w\  /w\  /w\  /w\  /w\  /w\  /w\  /|\  /w\  /w\  /w\  /w\  /w\  /

In [11]:
pipe = ExtractiveQAPipeline(reader, retriever)

In [12]:
simple_questions = [
    "How does your board oversee climate issues?",
    "Who is responsible for climate-related issues?",
    "How are climate issues monitored?",
    "Do you identify and respond to climate risks and opportunities?",
    "How do you define long-term time horizons?",
    "How do you identify and respond to climate risks and opportunities?",
    "Which climate risk types do you consider?",
    "Do you assess your portfolio's exposure to climate risks and opportunities?",
    "How do you assess your portfolio's exposure to climate risks and opportunities?",
    "Do you request climate-related information from your clients or investees?",
    "Do climate risks have a financial impact on your business?",
    "What climate risks may have a financial impact on your business?",
    "Are there any climate opportunities that have financial impact on your business?",
    "What climate opportunities will have a financial impact on your business?",
    "Have climate risks and opportunities influenced your strategy or finances?",
    "Do you analyze climate scenarios to inform your strategy?",
    "What climate scenarios do you analyze?",
    "How have climate risks and opportunities influenced your strategy?",
    "How have climate risks and opportunities influenced your financial planning?",
    "Do climate issues effect your external asset manager or independent asset manager selection?",
    "How do climate issues effect your external asset manager or independent asset manager selection?",
    "Did you have an emissions target?",
    "What is your absolute emissions target progress?",
    "What is your emissions intensity target progress?",
    "Did you have climate targets?",
    "What are your targets to increase low-carbon energy consumption or production?",
    "What are your methane-reduction targets?",
    "What were your buildings' operational Scope 1 emissions in metric tons of carbon dioxide?",
    "What were market-based power usage Scope 2 emissions in metric tons of carbon dioxide?",
    "What were your employees' Scope 3 emissions in metric tons of carbon dioxide?",
    "What climate metrics do you use?",
    "Do you analyze how your portfolio impacts the climate?",
    "What are your Scope 3 portfolio emissions?",
    "What is your Scope 3 portfolio impact from Category 15 Investments with alternative carbon footprinting or exposure metrics?",
    "Why do you not conduct analysis to understand how your portfolio impacts the climate with Scope 3 portfolio impact from Category 15 Investments with alternative carbon footprinting or exposure metrics?"
]

In [13]:
prediction = pipe.run(query=simple_questions[0], top_k_retriever=10, top_k_reader=3)

In [14]:
print_answers(prediction, details="minimal")

[   {'answer': 'Board diversity', 'context': 'Board diversity'},
    {'answer': 'Board diversity', 'context': 'Board diversity'},
    {'answer': '.', 'context': 'MAKE A DIFFERENCE.\nImpact your world.'}]


In [15]:
prediction["answers"][0]["answer"]

'Board diversity'

In [16]:
prediction["answers"][0]["context"]

'Board diversity'

In [17]:
prediction["answers"][0]

{'answer': 'Board diversity',
 'score': -5.725566387176514,
 'probability': 0.32834150069284124,
 'context': 'Board diversity',
 'offset_start': 0,
 'offset_end': 15,
 'offset_start_in_doc': 0,
 'offset_end_in_doc': 15,
 'document_id': 'ec012193-2234-47d1-bf11-7f422a122280',
 'meta': {'name': 'Walmart_2019_ESG_Report.pdf'}}

In [18]:
import wrapper_model
import mlflow
import torch

In [19]:
# reader.save("./saved_model/")

In [20]:
# mlflow_path = './pyfunc_model'

# mlflow.pyfunc.save_model(
#     path=mlflow_path,
#     loader_module=wrapper_model.__name__,
#     data_path="./saved_model/",
#     code_path=['./wrapper_model.py'],
#     conda_env='./conda.yml'
# )

In [ ]:
train_data = "./answers"
# train_data = "PATH/TO_YOUR/TRAIN_DATA" 
reader.train(data_dir=train_data, train_filename="answers.json", use_gpu=False, n_epochs=1, save_dir="tuned_1_iter_xlm-roberta-large-squad2")

03/08/2021 03:35:58 - INFO - farm.utils -   Using device: CPU 
03/08/2021 03:35:58 - INFO - farm.utils -   Number of GPUs: 0
03/08/2021 03:35:58 - INFO - farm.utils -   Distributed Training: False
03/08/2021 03:35:58 - INFO - farm.utils -   Automatic Mixed Precision: None
Preprocessing Dataset answers/answers.json: 100%|██████████| 4/4 [00:19<00:00,  4.95s/ Dicts]
03/08/2021 03:36:18 - ERROR - farm.data_handler.processor -   Unable to convert 1 samples to features. Their ids are : 2-59-17
03/08/2021 03:36:32 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
03/08/2021 03:36:32 - INFO - farm.modeling.optimization -   Loading optimizer `Trans

In [ ]:
mlflow_path = './pyfunc_model'

mlflow.pyfunc.save_model(
    path=mlflow_path,
    loader_module=wrapper_model.__name__,
    data_path="./tuned_1_iter_xlm-roberta-large-squad2/",
    code_path=['./wrapper_model.py'],
    conda_env='./conda.yml'
)